In [1]:
#!pip install langdetect


In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import re
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import string
from collections import Counter
from bs4 import BeautifulSoup
import seaborn as sns
from langdetect import DetectorFactory, detect


import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import PorterStemmer


nltk.download('stopwords')
nltk.download('punkt')

%matplotlib inline

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/saharawaji/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     /Users/saharawaji/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [3]:
wn = WordNetLemmatizer()
stop_words = set(stopwords.words('english'))

In [4]:
df_twcs = pd.read_csv("/Users/saharawaji/Documents/GitHub/Research/twcs 2.csv", on_bad_lines='skip')
df_twcs

,tweet_id,author_id,inbound,created_at,text,response_tweet_id,in_response_to_tweet_id
0,1,sprintcare,False,Tue Oct 31 22:10:47 +0000 2017,@115712 I understand. I would like to assist y...,2,3.0
1,2,115712,True,Tue Oct 31 22:11:45 +0000 2017,@sprintcare and how do you propose we do that,NaN,1.0
2,3,115712,True,Tue Oct 31 22:08:27 +0000 2017,@sprintcare I have sent several private messag...,1,4.0
3,4,sprintcare,False,Tue Oct 31 21:54:49 +0000 2017,@115712 Please send us a Private Message so th...,3,5.0
4,5,115712,True,Tue Oct 31 21:49:35 +0000 2017,@sprintcare I did.,4,6.0
...,...,...,...,...,...,...,...
2811769,2987947,sprintcare,False,Wed Nov 22 08:43:51 +0000 2017,"@823869 Hey, we'd be happy to look into this f...",NaN,2987948.0
2811770,2987948,823869,True,Wed Nov 22 08:35:16 +0000 2017,@115714 wtf!? I’ve been having really shitty s...,2987947,NaN
2811771,2812240,121673,True,Thu Nov 23 04:13:07 +0000 2017,@143549 @sprintcare You have to go to https://...,NaN,2812239.0
2811772,2987949,AldiUK,False,Wed Nov 22 08:31:24 +0000 2017,"@823870 Sounds delicious, Sarah! 😋 https://t.c...",NaN,2987950.0


In [5]:
column_names = df_twcs.columns
print(column_names)

Index(['tweet_id', 'author_id', 'inbound', 'created_at', 'text',
       'response_tweet_id', 'in_response_to_tweet_id'],
      dtype='object')


### Exploratory Data Analysis (EDA)

In [6]:
# Show information about the dataset
print(df_twcs.info())

#Summary statistics of numerical columns
print(df_twcs.describe())

# Check the first few rows of the dataset
print(df_twcs.head())


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2811774 entries, 0 to 2811773
Data columns (total 7 columns):
 #   Column                   Dtype  
---  ------                   -----  
 0   tweet_id                 int64  
 1   author_id                object 
 2   inbound                  bool   
 3   created_at               object 
 4   text                     object 
 5   response_tweet_id        object 
 6   in_response_to_tweet_id  float64
dtypes: bool(1), float64(1), int64(1), object(4)
memory usage: 131.4+ MB
None
           tweet_id  in_response_to_tweet_id
count  2.811774e+06             2.017439e+06
mean   1.504565e+06             1.463141e+06
std    8.616450e+05             8.665730e+05
min    1.000000e+00             1.000000e+00
25%    7.601652e+05             7.155105e+05
50%    1.507772e+06             1.439805e+06
75%    2.253296e+06             2.220646e+06
max    2.987950e+06             2.987950e+06
   tweet_id   author_id  inbound                      created_a

### Handling Missing Data:



In [7]:
# Drop rows with missing values
df_twcs.dropna(inplace=True)


In [8]:
# Impute missing values in the 'tweet_id' column with the mean
df_twcs['tweet_id'].fillna(df_twcs['tweet_id'].mean(), inplace=True)


In [9]:
# Count the number of missing values in each column
missing_values_count = df_twcs.isna().sum()


# print count of missing values for each column
print(missing_values_count)


tweet_id                   0
author_id                  0
inbound                    0
created_at                 0
text                       0
response_tweet_id          0
in_response_to_tweet_id    0
dtype: int64


In [10]:
df_twcs

,tweet_id,author_id,inbound,created_at,text,response_tweet_id,in_response_to_tweet_id
0,1,sprintcare,False,Tue Oct 31 22:10:47 +0000 2017,@115712 I understand. I would like to assist y...,2,3.0
2,3,115712,True,Tue Oct 31 22:08:27 +0000 2017,@sprintcare I have sent several private messag...,1,4.0
3,4,sprintcare,False,Tue Oct 31 21:54:49 +0000 2017,@115712 Please send us a Private Message so th...,3,5.0
4,5,115712,True,Tue Oct 31 21:49:35 +0000 2017,@sprintcare I did.,4,6.0
5,6,sprintcare,False,Tue Oct 31 21:46:24 +0000 2017,@115712 Can you please send us a private messa...,"5,7",8.0
...,...,...,...,...,...,...,...
2811718,2987897,VirginTrains,False,Wed Nov 22 06:57:43 +0000 2017,"@136417 Sorry to hear that James, did the the ...",2987898,2987899.0
2811728,2987907,nationalrailenq,False,Thu Nov 30 07:38:44 +0000 2017,London bound trains on the route between Broml...,2987906,2987908.0
2811736,2987915,CoxHelp,False,Thu Nov 30 07:27:11 +0000 2017,"@823858 Hello, this does not sound good. Can...",2987916,2987917.0
2811739,2987918,ArgosHelpers,False,Thu Nov 30 07:58:42 +0000 2017,@823859 Hi Natalie that doesn't sound good. Ca...,2987919,2987920.0


In [11]:
X = 2811774 - 976810

X

1834964

### Text Preprocessing:



In [12]:

# Tokenization
df_twcs['text'] = df_twcs['text'].apply(lambda x: word_tokenize(x.lower()))

# Stop word removal
stop_words = set(stopwords.words('english'))
df_twcs['text'] = df_twcs['text'].apply(lambda x: [word for word in x if word not in stop_words])

# Stemming (or Lemmatization)
stemmer = PorterStemmer()

df_twcs['text'] = df_twcs['text'].apply(lambda tokens: [word.lower() for word in tokens])


In [13]:
df_twcs['text']

0          [@, 115712, understand, ., would, like, assist...
2          [@, sprintcare, sent, several, private, messag...
3          [@, 115712, please, send, us, private, message...
4                                         [@, sprintcare, .]
5          [@, 115712, please, send, us, private, message...
                                 ...                        
2811718    [@, 136417, sorry, hear, james, ,, staff, made...
2811728    [london, bound, trains, route, bromley, south,...
2811736    [@, 823858, hello, ,, sound, good, ., dm, acco...
2811739    [@, 823859, hi, natalie, n't, sound, good, ., ...
2811760    [@, 783956, hi, ,, apologies, inconvenience, ....
Name: text, Length: 976810, dtype: object

# Feature Engineering:

1. Text Features:
2. Temporal Features:
3. Categorical Features:
4. Text-Based Features (NLP):
5. Interaction Features:
6. Domain-Specific Features:
7. Feature Scaling:
8. Feature Selection:


In [14]:
# 1. Text Features:
# A. Text Length: Create a feature that represents the length of the text in characters or words.
df_twcs['text_length'] = df_twcs['text'].apply(len)
print(df_twcs['text_length'])

# B. Word Count: Count the number of words in each text.
df_twcs['word_count'] = df_twcs['text'].apply(lambda tokens: len(tokens))

print(df_twcs['word_count'])


0          16
2           9
3          16
4           3
5          12
           ..
2811718    15
2811728    25
2811736    19
2811739    17
2811760    15
Name: text_length, Length: 976810, dtype: int64
0          16
2           9
3          16
4           3
5          12
           ..
2811718    15
2811728    25
2811736    19
2811739    17
2811760    15
Name: word_count, Length: 976810, dtype: int64


In [18]:
#2. Features of Temporal Order:

# A. Extracting Date and Time: Parse 'created_at' if it is in a string format to extract relevant temporal information such as month, day, hour, and so on.

#df_twcs['created_at'] = pd.to_datetime(df_twcs['created_at'])
df_twcs['created_at'] = pd.to_datetime(df_twcs['created_at'], utc=True)


df_twcs['year'] = df_twcs['created_at'].dt.year
df_twcs['month'] = df_twcs['created_at'].dt.month

# B. Time Since Tweet: Calculate the time elapsed since a tweet was posted relative to a reference date.
#reference_date = pd.Timestamp('2024-01-31')
reference_date = pd.Timestamp('2024-01-31', tz='UTC')
f_twcs['time_since_posted'] = (reference_date - df_twcs['created_at']).dt.total_seconds()




In [ ]:
# 3. Categorical Features:
# Encoding Categorical Variables: Convert categorical variables like 'author_id' into numerical values using techniques like one-hot encoding or label encoding.

df_twcs = pd.get_dummies(df_twcs, columns=['author_id'], prefix='author')
